# Exathlon (VLDB21) datasets

In [1]:
from typing import List, Optional
import matplotlib
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from config import data_raw_folder, data_processed_folder
from timeeval import Datasets
from timeeval.datasets import DatasetAnalyzer, DatasetRecord
from IPython.display import display, Markdown
import warnings

In [2]:
plt.rcParams["figure.figsize"] = (20, 10)

In [3]:
def find_datasets(folder):
    if not isinstance(folder, Path):
        folder = Path(folder)
    return sorted([f for d in source_folder.iterdir() if d.is_dir() for f in d.iterdir() if f.is_file()])

def plot_dataset(f, start_column_idx=0, end_column_idx=10):
    name = f.stem
    column_idxs = [0] + list(range(start_column_idx+1, end_column_idx))
    data = pd.read_csv(f).iloc[:, column_idxs].set_index("t")
    data.plot()
    
    # add anomaly labels
    try:
        anomalies = df_labels.loc[name]
    except KeyError:
        anomalies = pd.DataFrame([], columns=["begin", "end"])
    ax = plt.gca()
    for name, row in anomalies.iterrows():
        begin = row["begin"].astype(np.int_)
        end = row["end"].astype(np.int_)
        ax.add_patch(matplotlib.patches.Rectangle(
            (begin, data.min().min()),
            end-begin,
            data.max().max()-data.min().min(),
            color="yellow", alpha=0.75
        ))
    plt.legend()
    plt.title(name)
    return data

In [4]:
dataset_collection_name = "Exathlon"
source_folder = Path(data_raw_folder) / "exathlon" / "data" / "raw"
target_folder = Path(data_processed_folder)

print(f"Looking for source datasets in {Path(source_folder).absolute()} and\nsaving processed datasets in {Path(target_folder).absolute()}")

Looking for source datasets in /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-raw/exathlon/data/raw and
saving processed datasets in /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-processed


In [5]:
# shared by all datasets
dataset_type = "real"
input_type = "multivariate"
datetime_index = True
split_at = None

# create target directory
dataset_subfolder = Path(input_type) / dataset_collection_name
target_subfolder = target_folder / dataset_subfolder
target_subfolder.mkdir(parents=True, exist_ok=True)
print(f"Created directories {target_subfolder}")

dm = Datasets(target_folder)

Created directories /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-processed/multivariate/Exathlon


In [6]:
# load ground truth
df_labels = pd.read_csv(source_folder / "ground_truth.csv")

# we use extended end marker for anomalies
df_labels["extended_effect_end"].fillna(df_labels["root_cause_end"], axis=0, inplace=True)
df_labels.drop(columns=["root_cause_end", "trace_type", "anomaly_type", "anomaly_details"], inplace=True)
df_labels.columns = ["trace", "begin", "end"]
df_labels.set_index("trace", inplace=True)
df_labels

,begin,end
trace,,
2_1_100000_60,1.527535e+09,1.527536e+09
2_1_100000_60,1.527550e+09,1.527551e+09
2_1_100000_60,1.527564e+09,1.527565e+09
4_1_100000_61,1.527276e+09,1.527278e+09
4_1_100000_61,1.527291e+09,1.527293e+09
...,...,...
4_5_1000000_90,1.528967e+09,1.528967e+09
5_4_1000000_82,1.528983e+09,1.528983e+09
5_4_1000000_82,1.528987e+09,1.528987e+09


In [7]:
# load datasets and their metadata
datasets = find_datasets(source_folder)
datasets = dict((d.stem, d) for d in datasets)

df_datasets = pd.DataFrame({"filename": list(datasets.keys())}, columns=["filename", "app", "tpe", "rate"])
df_datasets.iloc[:, 1:] = df_datasets["filename"].str.split("_", expand=True).iloc[:, :3]
df_datasets.loc[:, ["app", "tpe", "rate"]] = df_datasets[["app", "tpe", "rate"]].astype(np.int_)
df_datasets = df_datasets.set_index(["app", "rate"]).sort_index()
df_datasets

filename tpe
app rate                        
1   10000       1_0_10000_17   0
    100000     1_0_100000_15   0
    100000     1_0_100000_16   0
    100000     1_2_100000_68   2
    500000     1_0_500000_18   0
...                      ...  ..
10  1000000  10_2_1000000_67   2
    1000000  10_3_1000000_75   3
    1000000  10_4_1000000_79   4
    1000000  10_5_1000000_85   5
    1200000   10_0_1200000_7   0

[93 rows x 2 columns]

In [12]:
# define preprocessing methods
def preprocess(dataset_name: Path, corr_threshold: float = 0.95, log_prefix: str = "  ", preselected_columns: Optional[List[str]] = None):
    exclude_columns = ["timestamp", "is_anomaly"]
    print(f"{log_prefix}reading dataset {dataset_name.name}")
    df = pd.read_csv(dataset_name)
    s_index = pd.Index(pd.to_datetime(df["t"], unit="s"), name="timestamp")
    df.drop(columns=["t"], inplace=True)
    
    print(f"{log_prefix}adding timestamp index")
    df.index = s_index
        
    # add labels
    print(f"{log_prefix}adding anomaly labels")
    df["is_anomaly"] = 0
    try:
        anomalies = df_labels.loc[dataset_name.stem:dataset_name.stem]
    except KeyError:
        anomalies = pd.DataFrame(columns=df_labels.columns, index=pd.Index([], name="trace"))
    for i, anomaly in anomalies.iterrows():
        begin = pd.to_datetime(anomaly["begin"], unit="s")
        end = pd.to_datetime(anomaly["end"], unit="s")
        df.loc[begin:end, "is_anomaly"] = 1
    
    if preselected_columns is not None:
        # use columns from test dataset (preselected)
        print(f"{log_prefix}reducing columns")
        columns = set(list(preselected_columns) + exclude_columns)
        unavailable_columns = columns - set(df.columns)
        if len(unavailable_columns) > 0:
            warnings.warn(f"Preselected columns contain column names that are not in the dataset ({unavailable_columns})")
        df = df[[c for c in columns if c not in unavailable_columns]]
    else:
        # columns with no real value
        print(f"{log_prefix}removing constant and empty columns")
        to_drop = [c for c in df.columns if c not in exclude_columns and (np.all(df[c].isin([0, -1])) or df[c].unique().shape[0] == 1)]
        df.drop(columns=to_drop, inplace=True)

        # only select correlated values
        print(f"{log_prefix}selecting with label correlated columns")
        s_corr = df.drop(columns=["is_anomaly"]).corrwith(df["is_anomaly"], drop=True).abs()
        cols = list(s_corr[s_corr > 0.5].index.values)
        df = df[cols + ["is_anomaly"]]

        # drop highly correlated columns
        print(f"{log_prefix}removing highly correlated columns")
        df_corr = df.drop(columns=["is_anomaly"]).corr().abs()
        df_corr = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(np.bool_))
        to_drop = [c for c in df_corr.columns if any(df_corr[c] > corr_threshold)]
        df.drop(columns=to_drop, inplace=True)
        
    # make timestamp index a column
    df.reset_index(drop=False, inplace=True)
    print(f"{log_prefix}final dataset shape: {df.shape}")

    return df


def transform_and_record(s_dataset: pd.Series,
                         s_partner: Optional[pd.Series] = None,
                         train_is_normal: bool = False,
                         train_type: str = "unsupervised"):
    if s_partner is None:
        train = False
        dataset_name = s_dataset["filename"]
    else:
        train = True
        dataset_name = s_dataset["filename"] + "-" + s_partner["filename"].split("_")[-1]

    test_filename = f"{dataset_name}.test.csv"
    test_path = dataset_subfolder / test_filename
    target_test_filepath = target_subfolder / test_filename
    target_meta_filepath = target_test_filepath.parent / f"{dataset_name}.{Datasets.METADATA_FILENAME_PREFIX}"
    
    if train:
        train_filename = f"{dataset_name}.train.csv"
        train_path = dataset_subfolder / train_filename
        target_train_filepath = target_subfolder / train_filename
    else:
        train_path = ""

    # Prepare test dataset
    print("  Preparing test dataset")
    f = datasets[s_dataset["filename"]]
    df_test = preprocess(f, log_prefix="  > ")
    test_columns = df_test.columns
    if set(test_columns) == set(["timestamp", "is_anomaly"]):
        warnings.warn(f"Found no columns that correlate with anomaly labels. Manual inspection needed for dataset {dataset_name}!")
        return
    df_test.to_csv(target_test_filepath, index=False)

    print("  Analyzing test metadata")
    da = DatasetAnalyzer((dataset_collection_name, dataset_name), is_train=False, df=df_test)
    da.save_to_json(target_meta_filepath, overwrite=True)
    meta = da.metadata

    # Prepare train dataset
    if train:
        print("  Preparing train dataset")
        f = datasets[s_partner["filename"]]
        df_train = preprocess(f, log_prefix="  > ", preselected_columns=test_columns)
        df_train.to_csv(target_train_filepath, index=False)

        print("  Analyzing train metadata")
        DatasetAnalyzer((dataset_collection_name, dataset_name), is_train=True, df=df_train)\
            .save_to_json(target_meta_filepath, overwrite=False)

    dm.add_dataset(DatasetRecord(
        collection_name=dataset_collection_name,
        dataset_name=dataset_name,
        train_path=train_path,
        test_path=test_path,
        dataset_type=dataset_type,
        datetime_index=datetime_index,
        split_at=split_at,
        train_type=train_type,
        train_is_normal=train_is_normal,
        input_type=input_type,
        length=meta.length,
        dimensions=meta.dimensions,
        contamination=meta.contamination,
        num_anomalies=meta.num_anomalies,
        min_anomaly_length=meta.anomaly_length.min,
        median_anomaly_length=meta.anomaly_length.median,
        max_anomaly_length=meta.anomaly_length.max,
        mean=meta.mean,
        stddev=meta.stddev,
        trend=meta.trend,
        stationarity=meta.get_stationarity_name(),
        period_size=np.nan
    ))
    print(f"Processed dataset {dataset_name}: {datasets[s_dataset['filename']]} -> {target_test_filepath}")

In [13]:
for _, s_dataset in df_datasets[df_datasets["tpe"] != 0].iterrows():
    dataset_name = s_dataset["filename"]
    df_partners = df_datasets.loc[s_dataset.name]
    df_partners = df_partners[(df_partners["filename"] != dataset_name) & (df_partners["tpe"].isin([s_dataset["tpe"], 0]))]
    
    if df_partners.shape[0] == 0:
        print(f"  {dataset_name} is unsupervised!")
        transform_and_record(s_dataset, train_is_normal=False, train_type="unsupervised")
        continue

    for _, s_partner in df_partners.iterrows():
        dataset_name = s_dataset["filename"] + "-" + s_partner["filename"].split("_")[-1]
        print(f"\n## Processing {dataset_name}")

        if s_partner["tpe"] == 0:
            print(f"  {dataset_name} is semi-supervised!")
            train_is_normal = True
            train_type = "semi-supervised"
        elif s_partner["tpe"] == s_dataset["tpe"]:
            print(f"  {dataset_name} is supervised!")
            train_is_normal = False
            train_type = "supervised"
        else:
            print(f"Found uncompatible partners:\ndataset={s_dataset}\npartner={s_partner}")
            continue

        transform_and_record(s_dataset, s_partner, train_is_normal=train_is_normal, train_type=train_type)

dm.save()


## Processing 1_2_100000_68-15
  1_2_100000_68-15 is semi-supervised!
  Preparing test dataset
  > reading dataset 1_2_100000_68.csv
  > adding timestamp index
  > adding anomaly labels
  > removing constant and empty columns
  > selecting with label correlated columns
  > removing highly correlated columns
  > final dataset shape: (2936, 46)
  Analyzing test metadata
  Preparing train dataset
  > reading dataset 1_0_100000_15.csv
  > adding timestamp index
  > adding anomaly labels
  > reducing columns
  > final dataset shape: (2682, 45)
  Analyzing train metadata


/tmp/ipykernel_27033/1691224462.py:29: UserWarning: Preselected columns contain column names that are not in the dataset ({'driver_benchmark_userclicks_1_20_10000_100000_batch076_1__StreamingMetrics_streaming_lastReceivedBatch_processingEndTime_value', 'timestamp'})
  warnings.warn(f"Preselected columns contain column names that are not in the dataset ({unavailable_columns})")
[('Exathlon', '1_2_100000_68-15') (train)] KPSS trend stationarity test for 3_NettyBlockTransfer_shuffle-server_usedDirectMemory_value encountered an error: cannot convert float NaN to integer
[('Exathlon', '1_2_100000_68-15') (train)] KPSS trend stationarity test for 3_executor_bytesWritten_count encountered an error: cannot convert float NaN to integer
[('Exathlon', '1_2_100000_68-15') (train)] KPSS trend stationarity test for 3_jvm_heap_committed_value encountered an error: cannot convert float NaN to integer
[('Exathlon', '1_2_100000_68-15') (train)] KPSS trend stationarity test for 3_jvm_heap_used_value enco

Processed dataset 1_2_100000_68-15: /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-raw/exathlon/data/raw/app1/1_2_100000_68.csv -> /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-processed/multivariate/Exathlon/1_2_100000_68-15.test.csv

## Processing 1_2_100000_68-16
  1_2_100000_68-16 is semi-supervised!
  Preparing test dataset
  > reading dataset 1_2_100000_68.csv
  > adding timestamp index
  > adding anomaly labels
  > removing constant and empty columns
  > selecting with label correlated columns
  > removing highly correlated columns
  > final dataset shape: (2936, 46)
  Analyzing test metadata
  Preparing train dataset
  > reading dataset 1_0_100000_16.csv
  > adding timestamp index
  > adding anomaly labels
  > reducing columns
  > final dataset shape: (3582, 45)
  Analyzing train metadata


/tmp/ipykernel_27033/1691224462.py:29: UserWarning: Preselected columns contain column names that are not in the dataset ({'driver_benchmark_userclicks_1_20_10000_100000_batch076_1__StreamingMetrics_streaming_lastReceivedBatch_processingEndTime_value', 'timestamp'})
  warnings.warn(f"Preselected columns contain column names that are not in the dataset ({unavailable_columns})")
[('Exathlon', '1_2_100000_68-16') (train)] KPSS trend stationarity test for 3_NettyBlockTransfer_shuffle-server_usedDirectMemory_value encountered an error: cannot convert float NaN to integer
[('Exathlon', '1_2_100000_68-16') (train)] KPSS trend stationarity test for 3_executor_bytesWritten_count encountered an error: cannot convert float NaN to integer
[('Exathlon', '1_2_100000_68-16') (train)] KPSS trend stationarity test for 3_jvm_heap_committed_value encountered an error: cannot convert float NaN to integer
[('Exathlon', '1_2_100000_68-16') (train)] KPSS trend stationarity test for 3_jvm_heap_used_value enco

Processed dataset 1_2_100000_68-16: /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-raw/exathlon/data/raw/app1/1_2_100000_68.csv -> /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-processed/multivariate/Exathlon/1_2_100000_68-16.test.csv

## Processing 1_4_1000000_80-14
  1_4_1000000_80-14 is semi-supervised!
  Preparing test dataset
  > reading dataset 1_4_1000000_80.csv
  > adding timestamp index
  > adding anomaly labels
  > removing constant and empty columns
  > selecting with label correlated columns
  > removing highly correlated columns
  > final dataset shape: (43087, 6)
  Analyzing test metadata
  Preparing train dataset
  > reading dataset 1_0_1000000_14.csv
  > adding timestamp index
  > adding anomaly labels
  > reducing columns
  > final dataset shape: (14347, 5)
  Analyzing train metadata


...

/tmp/ipykernel_27033/1691224462.py:87: UserWarning: Found no columns that correlate with anomaly labels. Manual inspection needed for dataset 8_5_1000000_83!
  warnings.warn(f"Found no columns that correlate with anomaly labels. Manual inspection needed for dataset {dataset_name}!")


  > adding timestamp index
  > adding anomaly labels
  > removing constant and empty columns
  > selecting with label correlated columns
  > removing highly correlated columns
  > final dataset shape: (46650, 9)
  Analyzing test metadata
Processed dataset 9_3_500000_74: /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-raw/exathlon/data/raw/app9/9_3_500000_74.csv -> /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-processed/multivariate/Exathlon/9_3_500000_74.test.csv
  9_2_1000000_66 is unsupervised!
  Preparing test dataset
  > reading dataset 9_2_1000000_66.csv
  > adding timestamp index
  > adding anomaly labels
  > removing constant and empty columns
  > selecting with label correlated columns
  > removing highly correlated columns
  > final dataset shape: (7481, 241)
  Analyzing test metadata
Processed dataset 9_2_1000000_66: /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-raw/exathlon/data/raw/app9/9_2_1000000_66.csv -> /home/se

/tmp/ipykernel_27033/1691224462.py:87: UserWarning: Found no columns that correlate with anomaly labels. Manual inspection needed for dataset 9_5_1000000_84!
  warnings.warn(f"Found no columns that correlate with anomaly labels. Manual inspection needed for dataset {dataset_name}!")


  > adding timestamp index
  > adding anomaly labels
  > removing constant and empty columns
  > selecting with label correlated columns
  > removing highly correlated columns
  > final dataset shape: (10250, 167)
  Analyzing test metadata
Processed dataset 10_2_1000000_67: /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-raw/exathlon/data/raw/app10/10_2_1000000_67.csv -> /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-processed/multivariate/Exathlon/10_2_1000000_67.test.csv
  10_3_1000000_75 is unsupervised!
  Preparing test dataset
  > reading dataset 10_3_1000000_75.csv
  > adding timestamp index
  > adding anomaly labels
  > removing constant and empty columns
  > selecting with label correlated columns
  > removing highly correlated columns
  > final dataset shape: (46656, 10)
  Analyzing test metadata
Processed dataset 10_3_1000000_75: /home/sebastian/Documents/Projects/akita/data/benchmark-data/data-raw/exathlon/data/raw/app10/10_3_1000000_75.c

/tmp/ipykernel_27033/1691224462.py:87: UserWarning: Found no columns that correlate with anomaly labels. Manual inspection needed for dataset 10_5_1000000_85!
  warnings.warn(f"Found no columns that correlate with anomaly labels. Manual inspection needed for dataset {dataset_name}!")


In [16]:
dm.refresh()
dm.df().loc[(slice(dataset_collection_name,dataset_collection_name), slice(None))]

train_path  \
collection_name dataset_name                                                           
Exathlon        10_2_1000000_67                                                  NaN   
                10_3_1000000_75                                                  NaN   
                10_4_1000000_79                                                  NaN   
                1_2_100000_68-15    multivariate/Exathlon/1_2_100000_68-15.train.csv   
                1_2_100000_68-16    multivariate/Exathlon/1_2_100000_68-16.train.csv   
                1_4_1000000_80-14  multivariate/Exathlon/1_4_1000000_80-14.train.csv   
                2_1_100000_60-20    multivariate/Exathlon/2_1_100000_60-20.train.csv   
                2_1_100000_60-22    multivariate/Exathlon/2_1_100000_60-22.train.csv   
                2_2_200000_69                                                    NaN   
                3_2_1000000_71                                                   NaN   
                3_2_500000_70                                                    NaN   
                4_1_100000_61-27    multivariate/Exathlon/4_1_100000_61-27.train.csv   
                4_1_100000_61-28    multivariate/Exathlon/4_1_100000_61-28.train.csv   
                4_1_100000_61-29    multivariate/Exathlon/4_1_100000_61-29.train.csv   
                4_1_100000_61-30    multivariate/Exathlon/4_1_100000_61-30.train.csv   
                4_1_100000_61-32    multivariate/Exathlon/4_1_100000_61-32.train.csv   
                5_1_100000_63-33    multivariate/Exathlon/5_1_100000_63-33.train.csv   
                5_1_100000_63-34    multivariate/Exathlon/5_1_100000_63-34.train.csv   
                5_1_100000_63-35    multivariate/Exathlon/5_1_100000_63-35.train.csv   
                5_1_100000_63-36    multivariate/Exathlon/5_1_100000_63-36.train.csv   
                5_1_100000_63-37    multivariate/Exathlon/5_1_100000_63-37.train.csv   
                5_1_100000_63-40    multivariate/Exathlon/5_1_100000_63-40.train.csv   
                5_1_100000_63-64    multivariate/Exathlon/5_1_100000_63-64.train.csv   
                5_1_100000_64-33    multivariate/Exathlon/5_1_100000_64-33.train.csv   
                5_1_100000_64-34    multivariate/Exathlon/5_1_100000_64-34.train.csv   
                5_1_100000_64-35    multivariate/Exathlon/5_1_100000_64-35.train.csv   
                5_1_100000_64-36    multivariate/Exathlon/5_1_100000_64-36.train.csv   
                5_1_100000_64-37    multivariate/Exathlon/5_1_100000_64-37.train.csv   
                5_1_100000_64-40    multivariate/Exathlon/5_1_100000_64-40.train.csv   
                5_1_100000_64-63    multivariate/Exathlon/5_1_100000_64-63.train.csv   
                5_1_500000_62                                                    NaN   
                5_2_1000000_72                                                   NaN   
                6_1_500000_65                                                    NaN   
                6_3_200000_76                                                    NaN   
                8_3_200000_73                                                    NaN   
                8_4_1000000_77                                                   NaN   
                9_2_1000000_66                                                   NaN   
                9_3_500000_74                                                    NaN   
                9_4_1000000_78                                                   NaN   

                                                                          test_path  \
collection_name dataset_name                                                          
Exathlon        10_2_1000000_67      multivariate/Exathlon/10_2_1000000_67.test.csv   
                10_3_1000000_75      multivariate/Exathlon/10_3_1000000_75.test.csv   
                10_4_1000000_79      multivariate/Exathlon/10_4_1000000_79.test.csv   
                1_2_100000_68-

Datasets for which our automatic extraction of relevant columns (dimensions/channels/variates) did not work properly:

- 1_5_1000000_86
- 2_5_1000000_87
- 2_5_1000000_88
- 3_4_1000000_81
- 3_5_1000000_89
- 4_5_1000000_90
- 5_4_1000000_82
- 5_5_1000000_91
- 5_5_1000000_92
- 6_5_1000000_93
- 8_5_1000000_83
- 9_5_1000000_84
- 10_5_1000000_85

These datasets require manual inspection and preprocessing!

## Exploration

In [ ]:
df_labels

In [ ]:
plot_dataset(datasets[list(datasets.keys())[23]], 110, 140)
plt.show()

In [ ]:
df = pd.read_csv(datasets[23])
print(f"Columns = {df.shape[1]}")
print(f"Rows = {df.shape[0]}")
df.describe()

In [ ]:
[c for c in df.columns]

In [ ]:
# drop highly correlated columns
df_corr = df.corr().abs()
df_corr = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(np.bool_))
df_corr

In [ ]:
threshold = 0.95
to_drop = [c for c in df_corr.columns if any(df_corr[c] > threshold)]
df2 = df.drop(columns=to_drop)
df2

In [ ]:
datasets = find_datasets(source_folder)
datasets = dict((d.stem, d) for d in datasets)
df_datasets = pd.DataFrame({"filename": list(datasets.keys())}, columns=["filename", "app", "tpe", "rate"])
df_datasets.iloc[:, 1:] = df_datasets["filename"].str.split("_", expand=True).iloc[:, :3]
df_datasets.loc[:, ["app", "tpe", "rate"]] = df_datasets[["app", "tpe", "rate"]].astype(np.int_)
res = df_datasets.groupby(by=["app", "rate", "tpe"]).count()
df_datasets = df_datasets.set_index(["app", "rate"]).sort_index()
res

In [ ]:
def preprocess(dataset_name: Path, corr_threshold: float = 0.95, log_prefix: str = "  ", preselected_columns: Optional[List[str]] = None):
    print(f"{log_prefix}reading dataset {dataset_name.name}")
    df = pd.read_csv(dataset_name)
    s_index = pd.Index(pd.to_datetime(df["t"], unit="s"), name="timestamp")
    df.drop(columns=["t"], inplace=True)
    
    print(f"{log_prefix}adding timestamp index")
    df.index = s_index
        
    # add labels
    print(f"{log_prefix}adding anomaly labels")
    df["is_anomaly"] = 0
    try:
        anomalies = df_labels.loc[dataset_name.stem:dataset_name.stem]
    except KeyError:
        anomalies = pd.DataFrame(columns=df_labels.columns, index=pd.Index([], name="trace"))
    for i, anomaly in anomalies.iterrows():
        begin = pd.to_datetime(anomaly["begin"], unit="s")
        end = pd.to_datetime(anomaly["end"], unit="s")
        df.loc[begin:end, "is_anomaly"] = 1
    
    if preselected_columns is not None:
        # use columns from test dataset (preselected)
        print(f"{log_prefix}reducing columns")
        columns = list(np.unique(list(preselected_columns) + ["is_anomaly"]))
        unavailable_columns = set(columns) - set(df.columns)
        if len(unavailable_columns) > 0:
            warnings.warn(f"Preselected columns contain column names that are not in the dataset ({unavailable_columns})")
        df = df[[c for c in columns if c not in unavailable_columns]]
    else:
        # columns with no real value
        print(f"{log_prefix}removing constant and empty columns")
        to_drop = [c for c in df.columns if c != "is_anomaly" and (np.all(df[c].isin([0, -1])) or df[c].unique().shape[0] == 1)]
        df.drop(columns=to_drop, inplace=True)

        # only select correlated values
        print(f"{log_prefix}selecting with label correlated columns")
        s_corr = df.drop(columns=["is_anomaly"]).corrwith(df["is_anomaly"], drop=True).abs()
        cols = list(s_corr[s_corr > 0.5].index.values)
        df = df[cols + ["is_anomaly"]]

        # drop highly correlated columns
        print(f"{log_prefix}removing highly correlated columns")
        df_corr = df.drop(columns=["is_anomaly"]).corr().abs()
        df_corr = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(np.bool_))
        to_drop = [c for c in df_corr.columns if any(df_corr[c] > corr_threshold)]
        df.drop(columns=to_drop, inplace=True)
        
    # make timestamp index a column
    df.reset_index(drop=False, inplace=True)
    print(f"{log_prefix}final dataset shape: {df.shape}")

    return df
    

df = preprocess(datasets[list(datasets.keys())[24]])
df

In [ ]:
df_corr = df.drop(columns=["is_anomaly"]).corrwith(df["is_anomaly"], drop=True).abs()
corr_cols = df_corr[df_corr > 0.5]#.index.values
corr_cols.sort_values()

In [ ]:
df[["driver_BlockManager_memory_memUsed_MB_value"]].plot()
#plt.gca().set_ylim(0, 20)
#plt.gca().set_xlim(pd.to_datetime(1.527534e+09, unit="s"), pd.to_datetime(1.527537e+09, unit="s"))
plt.gca().get_legend().remove()
plt.show()